In [1]:
import findspark
import os

findspark.init(os.environ['SPARK_HOME'])

In [2]:
from pyspark.sql import SparkSession, Window
import pyspark.sql.functions as F

In [3]:
spark = SparkSession.builder.appName('groupby-n-window').getOrCreate()

In [4]:
!ls ./data/stocks/

AAPL.csv
GOOG.csv
IBM.csv
INTC.csv
MSFT.csv


In [5]:
df_apple = spark.read.csv('./data/stocks/AAPL.csv', header=True, sep=',', inferSchema=True)
df_google = spark.read.csv('./data/stocks/GOOG.csv', header=True, sep=',', inferSchema=True)
df_ibm = spark.read.csv('./data/stocks/IBM.csv', header=True, sep=',', inferSchema=True)
df_intel = spark.read.csv('./data/stocks/INTC.csv', header=True, sep=',', inferSchema=True)
df_ms = spark.read.csv('./data/stocks/MSFT.csv', header=True, sep=',', inferSchema=True)

In [6]:
print("Apple: " + str(df_apple.count()))
print("Google: " + str(df_google.count()))
print("IBM: " + str(df_ibm.count()))
print("Intel: " + str(df_intel.count()))
print("Microsoft: " + str(df_ms.count()))

Apple: 253
Google: 253
IBM: 253
Intel: 253
Microsoft: 253


In [7]:
df_ibm.show(5)

+-------------------+----------+----------+----------+----------+----------+-------+
|               Date|      Open|      High|       Low|     Close| Adj Close| Volume|
+-------------------+----------+----------+----------+----------+----------+-------+
|2019-04-25 00:00:00|139.699997|    139.75|137.710007|138.630005|132.443527|2910100|
|2019-04-26 00:00:00|139.339996|139.889999|138.809998|139.440002|133.217377|2319800|
|2019-04-29 00:00:00|139.149994|139.630005|138.809998|139.050003|132.844788|2958300|
|2019-04-30 00:00:00|139.110001|140.440002|138.679993|140.270004|134.010345|4638800|
|2019-05-01 00:00:00|140.550003|141.809998|140.169998|140.559998|134.287384|3053700|
+-------------------+----------+----------+----------+----------+----------+-------+
only showing top 5 rows



In [11]:
# lit(): assign a constant value to a new column
df_apple = df_apple.withColumn('name', F.lit('AAPL'))
df_google = df_google.withColumn('name', F.lit('GOOG'))
df_ibm = df_ibm.withColumn('name', F.lit('IBM'))
df_intel = df_intel.withColumn('name', F.lit('INTC'))
df_ms = df_ms.withColumn('name', F.lit('MSFT'))

In [12]:
df_ibm.show(5)

+-------------------+----------+----------+----------+----------+----------+-------+----+
|               Date|      Open|      High|       Low|     Close| Adj Close| Volume|name|
+-------------------+----------+----------+----------+----------+----------+-------+----+
|2019-04-25 00:00:00|139.699997|    139.75|137.710007|138.630005|132.443527|2910100| IBM|
|2019-04-26 00:00:00|139.339996|139.889999|138.809998|139.440002|133.217377|2319800| IBM|
|2019-04-29 00:00:00|139.149994|139.630005|138.809998|139.050003|132.844788|2958300| IBM|
|2019-04-30 00:00:00|139.110001|140.440002|138.679993|140.270004|134.010345|4638800| IBM|
|2019-05-01 00:00:00|140.550003|141.809998|140.169998|140.559998|134.287384|3053700| IBM|
+-------------------+----------+----------+----------+----------+----------+-------+----+
only showing top 5 rows



In [13]:
df = df_apple.union(df_google).union(df_ibm).union(df_intel).union(df_ms)

In [14]:
df.count()

1265

## Group By

In [17]:
df.groupBy('name').agg(
    F.max(F.col('High')).alias('max'),
    F.min(F.col('Low')).alias('min'),
).withColumn(
    'diff', F.col('max') - F.col('min')
).withColumn(
    'fluctuate', F.round(F.col('diff') / F.col('min') * 100, 2)
).sort(F.desc('fluctuate')).show()

+----+-----------+-----------+------------------+---------+
|name|        max|        min|              diff|fluctuate|
+----+-----------+-----------+------------------+---------+
|AAPL| 327.850006| 170.270004|        157.580002|    92.55|
| IBM|     158.75|  90.559998|         68.190002|     75.3|
|INTC|  69.290001|  42.860001|26.430000000000007|    61.67|
|MSFT| 190.699997| 119.010002|         71.689995|    60.24|
|GOOG|1532.105957|1013.536011|        518.569946|    51.16|
+----+-----------+-----------+------------------+---------+



## Window

In [18]:
df.show(5)

+-------------------+----------+----------+----------+----------+----------+--------+----+
|               Date|      Open|      High|       Low|     Close| Adj Close|  Volume|name|
+-------------------+----------+----------+----------+----------+----------+--------+----+
|2019-04-25 00:00:00|206.830002|207.759995|205.119995|205.279999|202.627808|18543200|AAPL|
|2019-04-26 00:00:00|204.899994|     205.0|202.119995|204.300003|201.660461|18649100|AAPL|
|2019-04-29 00:00:00|204.399994|205.970001|203.860001|204.610001|201.966461|22204700|AAPL|
|2019-04-30 00:00:00|203.059998|203.399994|199.110001|200.669998|198.077362|46534900|AAPL|
|2019-05-01 00:00:00|209.880005|215.309998|209.229996|210.520004| 207.80011|64827300|AAPL|
+-------------------+----------+----------+----------+----------+----------+--------+----+
only showing top 5 rows



In [19]:
df.withColumn(
    'MovingAvg_5days_Close', 
    F.avg(F.col('Close')).over(
        Window.partitionBy('name').orderBy(F.desc('Date'))
              .rowsBetween(0, 4)
    ),
).show()

+-------------------+----------+----------+----------+----------+----------+--------+----+---------------------+
|               Date|      Open|      High|       Low|     Close| Adj Close|  Volume|name|MovingAvg_5days_Close|
+-------------------+----------+----------+----------+----------+----------+--------+----+---------------------+
|2020-04-24 00:00:00|277.200012| 283.01001|     277.0|282.970001|282.970001|31540300|AAPL|          275.8799988|
|2020-04-23 00:00:00|275.869995|    281.75|274.869995|275.029999|275.029999|31203600|AAPL|          275.8459962|
|2020-04-22 00:00:00|273.609985|277.899994|272.200012|276.100006|276.100006|29264300|AAPL|          278.1779968|
|2020-04-21 00:00:00|276.279999|    277.25|265.429993|268.369995|268.369995|45247900|AAPL|          279.8439942|
|2020-04-20 00:00:00|277.950012|281.679993|276.850006|276.929993|276.929993|32503800|AAPL|          283.5799928|
|2020-04-17 00:00:00|284.690002|286.950012|276.859985|282.799988|282.799988|53812500|AAPL|      

## Challenge

* Which stock's latest Adj Close has the largest rise compared to historical low?
* which stock's latest Adj Close has the largest fall compared to historical high?

In [36]:
df_1 = df.join(
    df.groupBy('name').agg(F.max(F.col('Date')).alias('Date')),
    how='inner',
    on=['name', 'Date'],
).select(['name', 'Adj Close']).withColumnRenamed('Adj Close', 'Recent')

df_2 = df.groupBy('name').agg(
    F.max(F.col('Adj Close')).alias('Highest'),
    F.min(F.col('Adj Close')).alias('Lowest'),
)

df_3 = df_1.join(df_2, how='inner', on='name') \
           .withColumn('Rise', F.col('Recent') - F.col('Lowest')) \
           .withColumn('Fall', F.col('Highest') - F.col('Recent'))

In [43]:
df_3.toPandas()

,name,Recent,Highest,Lowest,Rise,Fall
0,AAPL,282.970001,327.200012,171.719727,111.250274,44.230011
1,GOOG,1279.310059,1526.689941,1036.229980,243.080079,247.379882
2,IBM,124.720001,155.309998,94.769997,29.950004,30.589997
3,INTC,59.259998,68.134460,42.721867,16.538131,8.874462
4,MSFT,174.550003,188.185989,118.712952,55.837051,13.635986


In [40]:
df_3.sort(F.desc('Rise')).select(['name', 'Rise']).show(1)

+----+------------------+
|name|              Rise|
+----+------------------+
|GOOG|243.08007899999984|
+----+------------------+
only showing top 1 row



In [42]:
df_3.sort(F.desc('Fall')).select(['name', 'Fall']).show(1)

+----+------------------+
|name|              Fall|
+----+------------------+
|GOOG|247.37988200000018|
+----+------------------+
only showing top 1 row

